In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.component.feature_group_config as config
from comet_ml import Experiment

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

2025-02-03 17:39:26,964 INFO: Initializing external client
2025-02-03 17:39:26,966 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-03 17:39:28,138 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212591


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [5]:
data, _ = feature_view.training_data(
    description='Time-series hourly electricity demand values',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.91s) 


In [6]:
# drop `date` column
data.drop('seconds', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
data.sort_values(by=['sub_region_code', 'date'], inplace=True)
data

,date,sub_region_code,demand,temperature_2m
30502,2023-01-01 05:00:00+00:00,0,1477,14.6085
131572,2023-01-01 06:00:00+00:00,0,1428,14.4585
96878,2023-01-01 07:00:00+00:00,0,1392,14.4585
23544,2023-01-01 08:00:00+00:00,0,1366,14.6085
187100,2023-01-01 09:00:00+00:00,0,1353,14.7085
...,...,...,...,...
201727,2025-02-03 12:00:00+00:00,10,2191,8.7085
201768,2025-02-03 13:00:00+00:00,10,2292,9.1085
201776,2025-02-03 14:00:00+00:00,10,2203,9.2085
201778,2025-02-03 15:00:00+00:00,10,2065,8.9585


In [7]:
#transform the batch of data to features and target
from src.component.data_info import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    data,
    input_seq_len=24*28*1, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_demand_values_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 11/11 [00:05<00:00,  2.19it/s]

features_and_target.shape=(8459, 676)


In [8]:
features_and_target

,demand_previous_672_hour,demand_previous_671_hour,demand_previous_670_hour,demand_previous_669_hour,demand_previous_668_hour,demand_previous_667_hour,demand_previous_666_hour,demand_previous_665_hour,demand_previous_664_hour,demand_previous_663_hour,...,demand_previous_6_hour,demand_previous_5_hour,demand_previous_4_hour,demand_previous_3_hour,demand_previous_2_hour,demand_previous_1_hour,date,sub_region_code,temperature_2m,target_demand_values_next_hour
0,1477.0,1428.0,1392.0,1366.0,1353.0,1352.0,1361.0,1394.0,1429.0,1460.0,...,1786.0,1807.0,1776.0,1745.0,1732.0,1661.0,2023-01-29 05:00:00+00:00,0,0.558500,1599.0
1,1516.0,1450.0,1384.0,1351.0,1335.0,1334.0,1341.0,1375.0,1425.0,1486.0,...,1744.0,1796.0,1807.0,1789.0,1764.0,1719.0,2023-01-30 04:00:00+00:00,0,2.058500,1657.0
2,1664.0,1597.0,1530.0,1487.0,1455.0,1435.0,1437.0,1456.0,1510.0,1617.0,...,1910.0,1933.0,1983.0,1995.0,1968.0,1933.0,2023-01-31 03:00:00+00:00,0,2.558500,1874.0
3,1783.0,1718.0,1636.0,1549.0,1493.0,1459.0,1443.0,1440.0,1459.0,1516.0,...,1888.0,1862.0,1922.0,1994.0,2029.0,2004.0,2023-02-01 02:00:00+00:00,0,3.608500,1972.0
4,1816.0,1776.0,1731.0,1650.0,1556.0,1516.0,1481.0,1472.0,1471.0,1503.0,...,1882.0,1846.0,1873.0,1935.0,1994.0,2041.0,2023-02-02 01:00:00+00:00,0,3.858500,2017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8454,2067.0,2214.0,2268.0,2296.0,2306.0,2278.0,2200.0,2102.0,2168.0,2330.0,...,1841.0,1792.0,1772.0,1810.0,1935.0,2139.0,2025-01-30 13:00:00+00:00,10,7.508500,2269.0
8455,2096.0,2288.0,2425.0,2381.0,2270.0,2165.0,2096.0,2067.0,2101.0,2184.0,...,1826.0,1750.0,1710.0,1692.0,1726.0,1839.0,2025-01-31 12:00:00+00:00,10,5.808500,2029.0
8456,1939.0,1981.0,2071.0,2150.0,2203.0,2263.0,2334.0,2375.0,2369.0,2367.0,...,2057.0,1953.0,1882.0,1839.0,1817.0,1833.0,2025-02-01 11:00:00+00:00,10,10.408501,1882.0
8457,1775.0,1767.0,1788.0,1846.0,1905.0,1993.0,2111.0,2204.0,2277.0,2335.0,...,2168.0,2061.0,1966.0,1897.0,1851.0,1825.0,2025-02-02 10:00:00+00:00,10,7.758500,1832.0


In [9]:
features_and_target.date.min()

'2023-01-29 05:00:00+00:00'

In [10]:
features_and_target.date.max()

'2025-02-03 09:00:00+00:00'

In [11]:
#split the data
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.component.data_info import train_test_split


cutoff_date = pd.to_datetime(date.today() - timedelta(days=60), utc=True)

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_demand_values_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')
print(f"Training data range: {X_train['date'].min()} to {X_train['date'].max()}")
print(f"Testing data range: {X_test['date'].min()} to {X_test['date'].max()}")


cutoff_date=Timestamp('2024-12-05 00:00:00+0000', tz='UTC')
X_train.shape=(7766, 675)
y_train.shape=(7766,)
X_test.shape=(693, 675)
y_test.shape=(693,)
Training data range: 2023-01-29 05:00:00+00:00 to 2024-12-04 23:00:00+00:00
Testing data range: 2024-12-05 22:00:00+00:00 to 2025-02-03 09:00:00+00:00


In [12]:
x_tr=X_train.drop(['date'],axis=1)
x_ts=X_test.drop(['date'],axis=1)

In [13]:
#basleline model linear regrisson
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Create and train the linear regression model
model = LinearRegression()
model.fit(x_tr, y_train)

LinearRegression()

In [14]:
y_pred = model.predict(x_ts)
y_pred

array([1885.16590044, 1657.16114568, 1547.29081502, 1614.8559337 ,
       1905.97914142, 1760.67229769, 1877.67037636, 1876.83737683,
       1841.23352413, 1624.6099612 , 1517.63571406, 1553.90907084,
       1679.83022109, 1681.03942917, 1623.55128856, 1698.67708528,
       1610.74034899, 1623.35768053, 1616.2569723 , 1582.07748133,
       1764.18907289, 1803.41118896, 1852.83037119, 1816.79943578,
       1787.59258137, 1725.93393927, 1746.97827014, 1967.7761536 ,
       1974.61930249, 2020.82055486, 2021.55184271, 1767.22405258,
       1675.37559376, 1836.19642656, 1790.50841547, 1614.62225194,
       1588.02809992, 1613.82053266, 1545.75677081, 1590.19630901,
       1743.90828446, 1761.75850374, 1812.12286163, 1820.82890456,
       1887.19107884, 1974.92531337, 1982.81598926, 2005.51568872,
       2036.06665916, 1901.16466329, 1844.66170225, 1843.44150088,
       1925.65809518, 1734.75727648, 1768.31900503, 2019.86559609,
       2003.75119921, 1984.3076161 , 1942.05615437, 1793.98844

In [15]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
def evaluate_model(y_test, y_pred):
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_pred)
    return f"MAE is {test_mae:.4f} and MAPE is: {test_mape:.4f}"

In [16]:
evaluate_model(y_test, y_pred)

'MAE is 15.6190 and MAPE is: 0.0118'

In [17]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.component.model_info import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100), 
        #"learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True)
  
    }
       
    tss = KFold(n_splits=5)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [18]:
import warnings
warnings.filterwarnings("ignore")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=6)

[I 2025-02-03 17:39:53,619] A new study created in memory with name: no-name-57454293-e745-4601-963e-bc14482cdfee
  File "d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")
[I 2025-02-03 17:40:23,806] Trial 0 finished with value: 346.5973198713958 and parameters: {'num_leaves': 80, 'feature_fraction': 0.39574684453973386, 'bagging_fraction': 0.4518527995803667, 'min_child_samples': 20}. Best is trial 0 with value: 346.5973198713958.
[I 2025-02-03 17:41:31,768] Trial 1 finished with value: 346.34622195575076 and parameters: {'num_leaves': 197, 'feature_fraction': 0.5003312369921422, 'bagging_fraction': 0.5570866631309779, 'min_child_samples': 16}. Best is trial 1 with value: 346.34622195575076.
[I 2025-02-03 17:42:08,820] Trial 2 finished with value: 361.5271803500078 and parameters: {'num_leaves': 201, 'feature_fraction': 0.879874

In [19]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 234, 'feature_fraction': 0.720543786942808, 'bagging_fraction': 0.9070913988834302, 'min_child_samples': 7}


In [20]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train);

In [21]:
from src.component.model_info import evaluate_model
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=26.8245


In [22]:
#plot the result
from src.plot import plot_one_sample

plot_one_sample(
    example_id=1,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [23]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR /'LGB_model.pkl')

['D:\\Electricity_demand_predictor2\\models\\LGB_model.pkl']

In [24]:
#in order to save to model for model registoy we have to create scema first
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [25]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="electricity_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(str(MODELS_DIR /'LGB_model.pkl'))

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/2246354 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/4762 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/61601 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1212591/models/electricity_demand_predictor_next_hour/6


Model(name: 'electricity_demand_predictor_next_hour', version: 6)